In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, cumulative_trapezoid
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import scipy.constants as sc
from pathlib import Path
from numpy.linalg import eigvals

temperature = 300  # K

Angle = np.deg2rad(np.array([-14.92, -10.83, 30.79, -30.79, 10.83, 14.92, -14.91, -13.29, -53.16, 53.16, 13.29, 14.91]))
rotation = np.array([0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 2])
labels = {
    1: {'label': 'T-DPP-RIS', 'color': 'b'},
    2: {'label': 'T-T-RIS', 'color': 'm'},
    3: {'label': 'T-E-RIS', 'color': 'c'},
    # 4: {"label": "FT-FT", "color": 'g'},
    # 5: {"label": "ADTDI-FT", "color": 'r'},
}

In [2]:
def compute_ris_rotation_integrals(angles_deg, energies, temperature):
    """
    Compute rotation integrals for RIS model using discrete states.
    Returns:
    m_i, s_i: averaged cosine and sine values
    """
    kTval = sc.R * temperature / 1000
    angles_rad = np.deg2rad(angles_deg)
    
    # Calculate Boltzmann weights
    boltzmann_weights = np.exp(-energies / kTval)
    Z = np.sum(boltzmann_weights)  # Partition function
    
    # Calculate probability-weighted averages
    probabilities = boltzmann_weights / Z
    m_i = np.sum(probabilities * np.cos(angles_rad))
    s_i = np.sum(probabilities * np.sin(angles_rad))
    
    return m_i, s_i

def load_ris_data(data_label):
    """Load RIS data and return angles and energies"""
    file_name = Path(f"{data_label['label']}.txt")
    data = np.loadtxt(file_name)
    data = np.reshape(data, (-1, 2))
    data = np.unique(data, axis=0)
    return data[:, 0], data[:, 1]  # angles, energies



def make_Mmat_ris(all_ris_data, Angle_rad, rotation_types, temperature):
    """
    Create transformation matrix using RIS model.
    
    Parameters:
    all_ris_data: dict containing RIS data for each rotation type
    Angle_rad: array of bond angles in radians
    rotation_types: array of rotation type indices
    temperature: temperature in K
    """
    kTval = sc.R * temperature / 1000  # in kJ/mol
    M = len(rotation_types)
    A_list = []
    
    # Cache for rotation integrals (rot_id -> (m_i, s_i))
    integral_cache = {}
    
    for i in range(M):
        rot_id = int(rotation_types[i])
        theta = float(Angle_rad[i])
        
        if rot_id == 0:
            # Fixed rotation (no conformational flexibility)
            m_i, s_i = 1.0, 0.0
        else:
            if rot_id not in integral_cache:
                angles, energies = all_ris_data[rot_id]
                m_i, s_i = compute_ris_rotation_integrals(angles, energies, kTval)
                integral_cache[rot_id] = (m_i, s_i)
            else:
                m_i, s_i = integral_cache[rot_id]
        S = np.array([[1, 0.0, 0.0], 
                      [0.0, m_i, -s_i], 
                      [0.0, s_i, m_i]])
        # Bond angle rotation around z-axis
        c = np.cos(theta)
        s = np.sin(theta)
        R_z = np.array([[c, -s, 0.0], 
                        [s, c, 0.0], 
                        [0.0, 0.0, 1]])
        
        A_list.append(S @ R_z)

    Mmat = np.eye(3)
    for A in A_list:
        Mmat = A @ Mmat
    
    return Mmat


def compute_correlation(Mmat):
    """Calculate the persistence length in repeat units"""
    eigs = eigvals(Mmat)
    lambda_max = float(np.max(np.abs(eigs)))
    if lambda_max >= 1.0:
        return np.inf, lambda_max
    corr_length = -1.0 / np.log(lambda_max)
    return corr_length, lambda_max

In [3]:
all_data = {}
for key, label_info in labels.items():
    angles, energies = load_ris_data(label_info)
    all_data[key] = (angles, energies)
matrix = make_Mmat_ris(all_data, Angle, rotation, temperature)

In [4]:
lp_repeats, lam = compute_correlation(matrix)
# deflection = np.arccos(lam)

print(f"Max eigen value: lambda_max = {lam:.12f}")
print(f"Correlation length = {lp_repeats:.6f}")
# print(f"Deflection angle (in degrees) = {np.rad2deg(deflection):.6f}")

Max eigen value: lambda_max = 1.000000000000
Correlation length = inf
